In [1]:
import pandas as pd 
import numpy as np 

from sklearn import preprocessing
from sklearn import metrics

from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.combine import SMOTEENN

from imblearn.ensemble import BalancedBaggingClassifier

import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df= pd.read_csv("Training_Dataset.csv")

In [3]:
df.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51
0,1,Charge,0,106920,4,18,970,0.7969,0.1831,337236,...,0,0,0,0,0,0,1,0,0,0
1,2,Charge,2,0,4,19,1030,0.0678,0.0000,2918974,...,0,0,0,0,1,0,0,0,0,0
2,3,Charge,2,399168,4,35,327,0.7551,0.2986,498656,...,0,0,0,0,0,0,1,0,0,0
3,4,Charge,0,267300,4,274,1258,0.7969,0.5297,2480074,...,0,1,0,0,0,0,1,0,0,1
4,5,Charge,0,213840,6,1,0,2.3238,0.0000,2918974,...,0,0,0,0,1,0,0,1,0,0


In [4]:
spend = df.mvar36 + df.mvar37 + df.mvar38 + df.mvar39

In [5]:
income = df["mvar9"][df.mvar9>0]

In [6]:
income= income.sort_values(ascending=True).reset_index()

In [7]:
median_income= income.median()[1]

In [8]:
mean_income = income.mean()[1]

In [9]:
df.mvar9.replace(to_replace=0, value=mean_income, inplace=True)

In [10]:
df2=df
df2.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51
0,1,Charge,0,106920,4,18,970,0.7969,0.1831,337236.0,...,0,0,0,0,0,0,1,0,0,0
1,2,Charge,2,0,4,19,1030,0.0678,0.0000,2918974.0,...,0,0,0,0,1,0,0,0,0,0
2,3,Charge,2,399168,4,35,327,0.7551,0.2986,498656.0,...,0,0,0,0,0,0,1,0,0,0
3,4,Charge,0,267300,4,274,1258,0.7969,0.5297,2480074.0,...,0,1,0,0,0,0,1,0,0,1
4,5,Charge,0,213840,6,1,0,2.3238,0.0000,2918974.0,...,0,0,0,0,1,0,0,1,0,0


In [11]:
df2.drop(["cm_key", "mvar1", "mvar3"], axis=1, inplace=True)

In [12]:
df2a= df2.ix[:, 0:13]
df2a.head()

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15
0,0,4,18,970,0.7969,0.1831,337236.0,1,6.0899,Office supplies,12,1,0
1,2,4,19,1030,0.0678,0.0000,2918974.0,1,4.2016,Unknown,11,1,0
2,2,4,35,327,0.7551,0.2986,498656.0,0,3.3760,Misc services,12,1,1
3,0,4,274,1258,0.7969,0.5297,2480074.0,1,4.1082,Social services,12,1,0
4,0,6,1,0,2.3238,0.0000,2918974.0,0,0.0000,Unknown,6,2,0


In [13]:
df2b= df2.ix[:, 37:43]
df2b.head()

,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,1,0,0,1,0,0
4,0,0,0,0,0,0


In [14]:
df3= pd.concat([df2a, df2b], axis=1)

In [15]:
df3["spend"]=spend

In [16]:
# taking log
df3.mvar9 = np.log(df3.mvar9)

# normalizing
# std_scale = preprocessing.StandardScaler().fit(df3.mvar9)
# df3.mvar9 = std_scale.transform(df3.mvar9)

In [17]:
df3.spend = preprocessing.StandardScaler().fit_transform(df3.spend.reshape(-1, 1))

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [18]:
df3.mvar6 = preprocessing.StandardScaler().fit_transform(df3.mvar6.reshape(-1, 1))
# df3.mvar6 = std_scale.transform(df3.mvar6)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [18]:
df3.describe()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar13,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,spend
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,4.000000e+04
mean,1.117150,3.224150,89.264475,906.684950,0.726782,0.167726,13.500402,0.329925,3.384588,11.993125,1.046025,0.329175,0.293025,0.072300,0.070925,0.083075,0.025475,0.004925,-3.393119e-18
std,1.656335,2.324337,92.247710,921.135812,1.262927,0.206095,1.633287,0.470191,1.890469,3.274286,0.422328,0.772550,0.608663,0.315872,0.267014,0.320275,0.218237,0.070006,1.000013e+00
min,0.000000,0.000000,1.000000,0.000000,0.000100,0.000000,4.094345,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.669039e-01
25%,0.000000,2.000000,16.000000,327.000000,0.106000,0.000000,11.959003,0.000000,1.753100,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.462064e-01
50%,0.000000,3.000000,53.000000,586.000000,0.382000,0.115500,13.977593,0.000000,3.220350,12.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.125829e-01
75%,2.000000,4.000000,134.000000,1197.000000,0.796900,0.268200,14.886743,1.000000,5.011550,14.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.442737e-02
max,11.000000,12.000000,355.000000,4830.000000,10.000000,0.883100,17.347941,1.000000,7.001000,23.000000,9.000000,5.000000,5.000000,4.000000,3.000000,5.000000,4.000000,1.000000,1.101527e+02


In [19]:
le2 = preprocessing.LabelEncoder()
df3.mvar12 = le2.fit_transform(df3.mvar12)

In [20]:
df3["supplementary"] = df.mvar49
df3["elite"] = df.mvar50
df3["credit"] = df.mvar51

In [21]:
df3.elite = df3.elite*2
df3.credit = df3.credit*3

In [22]:
# 0:no card;  1:supplementary;  2:elite;  3:credit
df3["card_accepted"] = df3.supplementary + df3.credit + df3.elite

In [23]:
df3.drop(["supplementary", "elite", "credit"], axis=1, inplace=True)

In [24]:
df3.head()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,...,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,spend,card_accepted
0,0,4,18,970,0.7969,0.1831,12.728538,1,6.0899,12,...,1,0,0,0,0,0,0,0,0.321125,0
1,2,4,19,1030,0.0678,0.0000,14.886743,1,4.2016,17,...,1,0,0,0,0,0,0,0,-0.133348,0
2,2,4,35,327,0.7551,0.2986,13.119672,0,3.3760,10,...,1,1,0,0,0,0,0,0,-0.008518,0
3,0,4,274,1258,0.7969,0.5297,14.723799,1,4.1082,15,...,1,0,1,0,0,1,0,0,-0.072286,3
4,0,6,1,0,2.3238,0.0000,14.886743,0,0.0000,17,...,2,0,0,0,0,0,0,0,0.123923,1


In [25]:
x_df3 = df3.ix[:, :-1]
y_df3 = df3.ix[:, -1]

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [26]:
smote_enn = SMOTEENN(kind_smote='svm', k=5, ratio=0.4)

In [27]:
print np.bincount(y_df3)

[31122  3447  2654  2777]


In [28]:
x_df3_resampled, y_df3_resampled = smote_enn.fit_sample(x_df3, y_df3)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


In [29]:
print np.bincount(y_df3_resampled)

[12934  7820  9783  3275]


In [42]:
df4 = pd.Dataframe(data=x_df3_resampled[0:,0:],
                  columns=df3.columns)

AttributeError: 'module' object has no attribute 'Dataframe'

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x_df4, y_df4, test_size=0.3, random_state=42)

In [35]:
df3_majority = df3[df3.card_accepted==0]
df3_supp = df3[df3.card_accepted==1]
df3_elite = df3[df3.card_accepted==2]
df3_credit = df3[df3.card_accepted==3]

In [43]:
df3_majority_supp = pd.concat([df3_majority, df3_supp])
df3_majority_elite = pd.concat([df3_majority, df3_elite])
df3_majority_credit = pd.concat([df3_majority, df3_credit])


In [52]:
smote_enn = SMOTEENN(kind_smote='borderline1', k=5, ratio=1)

In [53]:
x_ms = df3_majority_supp.iloc[:, :-1]
y_ms = df3_majority_supp.iloc[:, -1]

In [54]:
x_msr, y_msr = smote_enn.fit_sample(x_ms, y_ms)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


In [55]:
print np.bincount(y_msr)

[19551 28271]


In [56]:
x_me = df3_majority_elite.iloc[:, :-1]
y_me = df3_majority_elite.iloc[:, -1]

In [57]:
x_mer, y_mer = smote_enn.fit_sample(x_me, y_me)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


In [58]:
print np.bincount(y_msr)

[19551 28271]


In [59]:
x_mc = df3_majority_credit.iloc[:, :-1]
y_mc = df3_majority_credit.iloc[:, -1]

x_mcr, y_mcr = smote_enn.fit_sample(x_mc, y_mc)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


In [60]:
print np.bincount(y_msr)

[19551 28271]


In [62]:
x_msr = pd.DataFrame(x_msr)

In [63]:
x_msr["card_accepted"] = y_msr

In [64]:
x_mer = pd.DataFrame(x_mer)

x_mer["card_accepted"] = y_mer

In [65]:
x_mcr = pd.DataFrame(x_mcr)

In [66]:
x_mcr["card_accepted"] = y_mcr

In [45]:
df3_supp_upsampled = resample(df3_supp,
                              replace=True,
                              n_samples=9000,
                              random_state=123)

In [111]:
df3_elite_upsampled = resample(df3_elite,
                               replace=True,
                               n_samples=9000,
                               random_state=123)

In [36]:
df3_credit_upsampled = resample(df3_credit,
                                replace=True,    
                                n_samples=7000,     
                                random_state=123)

In [67]:
df4 = pd.concat([x_msr, x_mer, x_mcr])
df4.shape

(148393, 21)

In [68]:
train, test= train_test_split(df4, test_size=0.3)

In [69]:
x_train = train.ix[:, :-1]
y_train = train.ix[:, -1]
x_test = test.ix[:, :-1]
y_test = test.ix[:, -1]

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
rfc = RandomForestClassifier(n_estimators=300, max_features='log2', oob_score=True)

In [134]:
rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True,
            class_weight={0: 0.1, 1: 0.3, 2: 0.3, 3: 0.3},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [135]:
y_predicted = rfc.predict(x_test)

In [136]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.884394447104
[[3772  221  290  170]
 [ 341 2245   19   18]
 [ 158    0 2564   17]
 [ 190   11   14 2504]]
             precision    recall  f1-score   support

          0       0.85      0.85      0.85      4453
          1       0.91      0.86      0.88      2623
          2       0.89      0.94      0.91      2739
          3       0.92      0.92      0.92      2719

avg / total       0.88      0.88      0.88     12534



In [300]:
param_grid = {
    "max_depth": [3, 5, 10, 100]
}

In [301]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid)

In [302]:
CV_rfc.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [303]:
print CV_rfc.best_params_
print CV_rfc.best_score_

{'max_depth': 100}
0.886439865836


In [132]:
rfc_tuned = RandomForestClassifier(n_estimators=200,
                                   max_features='sqrt',
                                    oob_score=True,
                                  )

In [133]:
rfc_tuned.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [134]:
y_predicted = rfc_tuned.predict(x_test)

In [135]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.93160070084
[[18315    36   124    61]
 [  726  7186   319   303]
 [  427    18  8308    74]
 [  522   157   278  7664]]
             precision    recall  f1-score   support

          0       0.92      0.99      0.95     18536
          1       0.97      0.84      0.90      8534
          2       0.92      0.94      0.93      8827
          3       0.95      0.89      0.92      8621

avg / total       0.93      0.93      0.93     44518



In [75]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=3,
    min_child_weight=1000,
    objective= 'multi:softmax',
 )

In [137]:
param_grid = {
    'n_estimators': [200, 300, 400],
    'learning_rate': [0.1, 0.01, 0.015]
}

In [138]:
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid)

In [76]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1000, missing=None,
       n_estimators=300, nthread=-1, objective='multi:softprob', seed=0,
       silent=True, subsample=1)

In [77]:
y_predicted = xgb.predict(x_test)

In [78]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.815984545577
[[18296    20   123    97]
 [ 1099  5315   931  1189]
 [  865   178  7152   632]
 [  956   738  1364  5563]]
             precision    recall  f1-score   support

          0       0.86      0.99      0.92     18536
          1       0.85      0.62      0.72      8534
          2       0.75      0.81      0.78      8827
          3       0.74      0.65      0.69      8621

avg / total       0.81      0.82      0.81     44518



In [ ]:
bbc = BalancedBagg

In [79]:
test_df= pd.read_csv("Leaderboard_Dataset.csv")
test_df.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,50001,Charge,2,0,4,18,460,0.2495,0.2310,180512,...,32886.89,11345.47,7914.21,7704.62,0,0,0,0,0,0
1,50002,Charge,2,0,0,45,381,1.9329,0.7196,11295211,...,104722.52,255544.59,116216.40,152673.00,1,0,0,0,0,0
2,50003,Charge,0,0,4,3,0,0.7969,0.0000,45161,...,19689.03,0.00,0.00,0.00,0,0,0,0,0,0
3,50004,Charge,1,18711,9,89,432,0.1016,0.5074,420972,...,13154.43,7671.62,3357.24,2651.22,0,1,0,0,0,0
4,50005,Charge,0,21384,4,7,0,0.7969,0.2310,880618,...,32071.78,15376.01,14390.97,0.00,0,0,0,0,0,0


In [80]:
test_income = test_df["mvar9"][test_df.mvar9>0]
test_income= test_income.sort_values(ascending=True).reset_index()
median_income= test_income.median()[1]
mean_income = test_income.mean()[1]
test_df.mvar9.replace(to_replace=0, value=mean_income, inplace=True)
test_spend = test_df.mvar36 + test_df.mvar37 + test_df.mvar38 + test_df.mvar39

In [81]:
test_df.drop(["cm_key", "mvar1", "mvar3"], axis=1, inplace=True)

In [82]:
test_df.head()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,...,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,2,4,18,460,0.2495,0.2310,180512.0,0,5.0612,Personal services,...,32886.89,11345.47,7914.21,7704.62,0,0,0,0,0,0
1,2,0,45,381,1.9329,0.7196,11295211.0,0,4.0930,Misc services,...,104722.52,255544.59,116216.40,152673.00,1,0,0,0,0,0
2,0,4,3,0,0.7969,0.0000,45161.0,0,2.3941,Misc services,...,19689.03,0.00,0.00,0.00,0,0,0,0,0,0
3,1,9,89,432,0.1016,0.5074,420972.0,0,1.5442,Consumables,...,13154.43,7671.62,3357.24,2651.22,0,1,0,0,0,0
4,0,4,7,0,0.7969,0.2310,880618.0,0,2.6261,Non durable goods,...,32071.78,15376.01,14390.97,0.00,0,0,0,0,0,0


In [83]:
test_dfa= test_df.ix[:,0:13]

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [84]:
test_dfb= test_df.ix[:, 37:43]

In [85]:
test_df= pd.concat([test_dfa, test_dfb], axis=1)

In [86]:
le = preprocessing.LabelEncoder()
test_df.mvar12 = le.fit_transform(test_df.mvar12)

In [87]:
test_df.head(2)

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45
0,2,4,18,460,0.2495,0.2310,180512.0,0,5.0612,13,12,1,0,0,0,0,0,0,0
1,2,0,45,381,1.9329,0.7196,11295211.0,0,4.0930,10,13,1,0,1,0,0,0,0,0


In [88]:
test_df["mvar9"]= np.log(test_df["mvar9"])
# std_scale = preprocessing.StandardScaler().fit(test_df.mvar9)
# test_df.mvar9 = std_scale.transform(test_df.mvar9)

In [89]:
test_df["spend"]=test_spend

In [90]:
df.spend = preprocessing.StandardScaler().fit(test_df.spend.reshape(-1, 1))
# test_df.spend = std_scale.transform(test_df.spend)

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [91]:
df.mvar6 = preprocessing.StandardScaler().fit(test_df.mvar6.reshape(-1, 1))

/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/media/pindaari/Softwares/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [92]:
test_df.head()

,mvar2,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar11,mvar12,mvar13,mvar14,mvar15,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,spend
0,2,4,18,460,0.2495,0.2310,12.103553,0,5.0612,13,12,1,0,0,0,0,0,0,0,59851.19
1,2,0,45,381,1.9329,0.7196,16.239889,0,4.0930,10,13,1,0,1,0,0,0,0,0,629156.51
2,0,4,3,0,0.7969,0.0000,10.717989,0,2.3941,10,10,1,0,0,0,0,0,0,0,19689.03
3,1,9,89,432,0.1016,0.5074,12.950322,0,1.5442,3,11,1,1,0,1,0,0,0,0,26834.51
4,0,4,7,0,0.7969,0.2310,13.688379,0,2.6261,11,14,6,0,0,0,0,0,0,0,61838.76


In [136]:
test_result= rfc_tuned.predict(test_df)

In [137]:
test_result

array([0, 0, 0, ..., 0, 0, 0])

In [138]:
test_df2 = pd.read_csv("Leaderboard_Dataset.csv")

In [139]:
result= pd.DataFrame(
{
    "card_accepted": test_result,
    "cm_key": test_df2.cm_key
    
})

In [140]:
result.shape

(10000, 2)

In [141]:
result.card_accepted.replace(to_replace=0, value="No_card", inplace=True)

In [142]:
result.card_accepted.replace(to_replace=1, value="Supp", inplace=True)
result.card_accepted.replace(to_replace=2, value="Elite", inplace=True)
result.card_accepted.replace(to_replace=3, value="Credit", inplace=True)

In [143]:
submission= result[result.card_accepted != "No_card"]

In [144]:
# submission = result[result.card_accepted != "Credit"]

In [145]:
submission.shape

(413, 2)

In [192]:
submission= submission.reindex_axis(sorted(submission.columns, reverse=True), axis=1)

In [193]:
submission= submission[:1000]

In [194]:
submission.head()

,cm_key,card_accepted
4,50005,Elite
15,50016,Elite
51,50052,Elite
53,50054,Elite
58,50059,Elite


In [195]:
submission.to_csv("Reckless_Serenade_IITRoorkee_38.csv", header=False, index=False)